In [1]:
!wget -O raw.csv -nc "http://www.belpa.be/pub/belpa.csv?piv_naam=&pin_begr_jr=&piv_postcode=&piv_gemeente=&piv_subtype_bedrag=&piv_type_bedrag=&pin_ondergrens=&pin_bovengrens=&pin_pagesize=10&pin_page_start=1&piv_sort=sort_webnaam&pin_sort_dir=0&piv_lng=EN&pin_inprod=1"        

--2017-05-17 20:39:17--  http://www.belpa.be/pub/belpa.csv?piv_naam=&pin_begr_jr=&piv_postcode=&piv_gemeente=&piv_subtype_bedrag=&piv_type_bedrag=&pin_ondergrens=&pin_bovengrens=&pin_pagesize=10&pin_page_start=1&piv_sort=sort_webnaam&pin_sort_dir=0&piv_lng=EN&pin_inprod=1
Resolving www.belpa.be... 195.244.170.200
Connecting to www.belpa.be|195.244.170.200|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/vnd.ms-excel]
Saving to: ‘raw_new.csv’

raw_new.csv             [           <=>      ]  47.13M  5.64MB/s    in 9.0s    

2017-05-17 20:40:07 (5.26 MB/s) - ‘raw_new.csv’ saved [49420693]



In [4]:
import pandas as pd

In [5]:
df_raw = pd.read_csv('raw.csv', delimiter=';', encoding='utf-8')
df_raw.head()

,name,postal_code,municipality,eagf,eafr,total,year,I.1,I.2,I.3,...,V/B.4.2,V/B.4.3,V/B.4.4,V/B.4.5,V/B.5,V/B.6,VI/A.1,VI/B.1,VI/C.1,VI/D.1
0,ATHENEE ROYAL,7000,Mons,"5.682,07","0,00","5.682,07",2015,"0,00","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
1,VILLE DE FONTAINE-L EVEQUE COMMUNES,6140,Fontaine-l'Evêque,"2.817,50","0,00","2.817,50",2015,"0,00","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
2,"AERO- UND MODELLCLUB ""FEUERVOGEL"" BULLINGEN VOG",4760,Büllingen,"0,00","864,00","864,00",2015,"0,00","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
3,"GEMEENTELIJKE BASISSCHOOL GEMEENTESCHOOL ""DE P...",9280,Lebbeke,"744,00","0,00","744,00",2015,"0,00","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
4,"GROUPE D ACTION LOCALE ""PAYS DES CONDRUSES"" ASBL",4577,Modave,"0,00","185.391,56","185.391,56",2015,"0,00","0,00","0,00",...,"55.429,89","63.520,20","11.798,59","33.851,09","0,00","0,00","0,00","0,00","0,00","0,00"


In [6]:
id_vars = 'name	postal_code	municipality	eagf	eafr	total	year'.split('\t')
all_columns = list(df_raw.columns)
total_cols = ['total', 'eafr', 'eagf']
value_vars = list(set(all_columns) - set(id_vars))

In [7]:
def fix_nums(series):
    return series.astype(str).str.replace('.', '').str.replace(',', '.')

for val in value_vars + total_cols:
    df_raw[val] = pd.to_numeric(fix_nums(df_raw[val]), errors='coerce')
    
df_raw.head()

,name,postal_code,municipality,eagf,eafr,total,year,I.1,I.2,I.3,...,V/B.4.2,V/B.4.3,V/B.4.4,V/B.4.5,V/B.5,V/B.6,VI/A.1,VI/B.1,VI/C.1,VI/D.1
0,ATHENEE ROYAL,7000,Mons,5682.07,0.00,5682.07,2015,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
1,VILLE DE FONTAINE-L EVEQUE COMMUNES,6140,Fontaine-l'Evêque,2817.50,0.00,2817.50,2015,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2,"AERO- UND MODELLCLUB ""FEUERVOGEL"" BULLINGEN VOG",4760,Büllingen,0.00,864.00,864.00,2015,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
3,"GEMEENTELIJKE BASISSCHOOL GEMEENTESCHOOL ""DE P...",9280,Lebbeke,744.00,0.00,744.00,2015,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4,"GROUPE D ACTION LOCALE ""PAYS DES CONDRUSES"" ASBL",4577,Modave,0.00,185391.56,185391.56,2015,0.0,0.0,0.0,...,55429.89,63520.2,11798.59,33851.09,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df = pd.melt(df_raw, id_vars=id_vars, value_vars=value_vars, value_name='amount', var_name='scheme')
df = df[df['amount'] > 0]
df = df.drop(total_cols, 1)
df.head()

,name,postal_code,municipality,year,scheme,amount
152,ABINET JOSEPH-KAISER JOSETTE EP.,6600,Bastogne,2014,IV/A.18,2346.00
164,ABSHOFF CEDRIC,4890,Thimister-Clermont,2015,IV/A.18,1736.00
165,ABSHOFF CEDRIC,4890,Thimister-Clermont,2014,IV/A.18,1736.00
167,ABSHOFF JEROME,4860,Pepinster,2014,IV/A.18,1914.18
182,ACHENNE ANDRE-MARIE,6681,Sainte-Ode,2015,IV/A.18,2346.00


In [9]:
df = df.rename(columns={
    'name': 'recipient_name',
    'postal_code': 'recipient_postcode',
    'municipality': 'recipient_location',
})
df['currency'] = 'EUR'
df['country'] = 'BE'

def generate_id(row):
    return u'BE-%s-%s' % (row['recipient_postcode'], row['recipient_name'])

df['recipient_id'] = df.apply(generate_id, 1)
df.head()

,recipient_name,recipient_postcode,recipient_location,year,scheme,amount,currency,country,recipient_id
152,ABINET JOSEPH-KAISER JOSETTE EP.,6600,Bastogne,2014,IV/A.18,2346.00,EUR,BE,BE-6600-ABINET JOSEPH-KAISER JOSETTE EP.
164,ABSHOFF CEDRIC,4890,Thimister-Clermont,2015,IV/A.18,1736.00,EUR,BE,BE-4890-ABSHOFF CEDRIC
165,ABSHOFF CEDRIC,4890,Thimister-Clermont,2014,IV/A.18,1736.00,EUR,BE,BE-4890-ABSHOFF CEDRIC
167,ABSHOFF JEROME,4860,Pepinster,2014,IV/A.18,1914.18,EUR,BE,BE-4860-ABSHOFF JEROME
182,ACHENNE ANDRE-MARIE,6681,Sainte-Ode,2015,IV/A.18,2346.00,EUR,BE,BE-6681-ACHENNE ANDRE-MARIE


In [10]:
df.year.value_counts()

2015    123282
2014     87489
Name: year, dtype: int64

In [13]:
df.to_csv('be_2014_2015.csv.gz', index=False, encoding='utf-8', compression='gzip')